# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
import nltk
nltk.download(['punkt','stopwords','wordnet','omw-1.4'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arwao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arwao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arwao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\arwao\AppData\Roaming\nltk_data...


True

In [34]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

In [4]:
# load data from database
conn = create_engine('sqlite:///Data scientist - NanoDegree//Disaster Response pipeline//responsesDataset.db')
df = pd.read_sql('SELECT * FROM Responses', conn)
df

,id,message,original,genre,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message'] 
Y = df.iloc[:, 4:] #labels

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    """
    Description: This function will tokenize, 
    clean the text, and return words to its roots.
  
    Parameters:
    text (str): the function will recive text in str format.
    
    Returns:
    str: clean and tokenized text. 
    
    """
    # normalize text
    text = text.lower() 
    #clean text
    text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
    #tokenize text using this function
    words = word_tokenize(text) 
    #store stop words 
    stop_words = stopwords.words('english') 
    #remove all stop words
    new_text = [x for x in words if x not in stop_words]
    #stemming
    stemmed = [PorterStemmer().stem(x) for x in new_text]
    #lemming
    lemmed = [WordNetLemmatizer().lemmatize(x) for x in stemmed]
    return lemmed

In [10]:
#testing the function
tokenize('Hello from the other side!!')

['hello', 'side']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
#switching warnings off since they are a bit annoying :)
import warnings
warnings.filterwarnings('ignore')

In [12]:
#pipeline for features extraction, transformation, classification 


pipeline1 = Pipeline([
        ('vectorize', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('KNN',  MultiOutputClassifier(KNeighborsClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
#train and test sets with 70% for train and 30% for test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [14]:
#fitting the training set by passing it to the pipeline
pipeline1.fit(X_train, y_train)

Pipeline(steps=[('vectorize',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000237C82F85E0>)),
                ('tfidf', TfidfTransformer()),
                ('KNN',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
 score = pipeline1.score(X_test, y_test)
 score 

0.26573426573426573

In [15]:
#storing predictions to test performance
y_pred = pipeline1.predict(X_test)

In [16]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [27]:
y_test

,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
13763,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
21716,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16002,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3745,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12096,1,0,0,1,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11033,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8166,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
719,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
y_test.shape[1]

36

In [13]:
#Y

In [18]:
def model_score(y_true, y_pred):
    """
    Compute the classification report for each column of y_true and y_pred.
    
    Parameters
    ----------
    y_true : array-like of shape (n_samples, n_classes)
        True labels.
    y_pred : array-like of shape (n_samples, n_classes)
        Predicted labels.
        
    Returns
    -------
    None
    """
    #y_pred = pipeline.predict(X_test)
    for i in range(y_true.shape[1]):
        print(f"Column {i}:")
        print(classification_report(y_true[:, i], y_pred[:, i]))
        print('\n')

In [19]:
type(y_pred)

numpy.ndarray

In [20]:
type(y_test) #must be array to handle any error

pandas.core.frame.DataFrame

In [21]:
model_score(y_test.to_numpy(), y_pred)

Column 0:
              precision    recall  f1-score   support

           0       0.57      0.32      0.41      1787
           1       0.82      0.92      0.87      6028
           2       0.25      0.36      0.29        50

    accuracy                           0.78      7865
   macro avg       0.55      0.54      0.52      7865
weighted avg       0.76      0.78      0.76      7865



Column 1:
              precision    recall  f1-score   support

           0       0.83      0.99      0.91      6481
           1       0.75      0.08      0.14      1384

    accuracy                           0.83      7865
   macro avg       0.79      0.54      0.52      7865
weighted avg       0.82      0.83      0.77      7865



Column 2:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline1.get_params()

{'memory': None,
 'steps': [('vectorize',
   CountVectorizer(tokenizer=<function tokenize at 0x00000237C82F85E0>)),
  ('tfidf', TfidfTransformer()),
  ('KNN', MultiOutputClassifier(estimator=KNeighborsClassifier()))],
 'verbose': False,
 'vectorize': CountVectorizer(tokenizer=<function tokenize at 0x00000237C82F85E0>),
 'tfidf': TfidfTransformer(),
 'KNN': MultiOutputClassifier(estimator=KNeighborsClassifier()),
 'vectorize__analyzer': 'word',
 'vectorize__binary': False,
 'vectorize__decode_error': 'strict',
 'vectorize__dtype': numpy.int64,
 'vectorize__encoding': 'utf-8',
 'vectorize__input': 'content',
 'vectorize__lowercase': True,
 'vectorize__max_df': 1.0,
 'vectorize__max_features': None,
 'vectorize__min_df': 1,
 'vectorize__ngram_range': (1, 1),
 'vectorize__preprocessor': None,
 'vectorize__stop_words': None,
 'vectorize__strip_accents': None,
 'vectorize__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vectorize__tokenizer': <function __main__.tokenize(text)>,
 'vectorize__vocabular

In [25]:
parameters = {
    'KNN__estimator__n_neighbors': [5,7,10,15,20]
}

cv = GridSearchCV(pipeline1, parameters)
cv

GridSearchCV(estimator=Pipeline(steps=[('vectorize',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000237C82F85E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('KNN',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             param_grid={'verbose': [False, True]})

In [24]:
grid_search= cv.fit(X_train, y_train)
print(grid_search.best_params_)

{'KNN__estimator__n_neighbors': 10}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
#testing the model using best params, k = 10
pipeline1 = Pipeline([
        ('vectorize', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('KNN',  MultiOutputClassifier(KNeighborsClassifier(n_neighbors = 10)))
    ])

pipeline1.fit(X_train, y_train)
y_pred = pipeline1.predict(X_test)
model_score(y_test.to_numpy(), y_pred)

Column 0:
              precision    recall  f1-score   support

           0       0.57      0.45      0.51      1787
           1       0.84      0.90      0.87      6028
           2       0.50      0.10      0.17        50

    accuracy                           0.79      7865
   macro avg       0.64      0.48      0.51      7865
weighted avg       0.78      0.79      0.78      7865



Column 1:
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      6481
           1       0.85      0.22      0.35      1384

    accuracy                           0.86      7865
   macro avg       0.85      0.61      0.64      7865
weighted avg       0.86      0.86      0.82      7865



Column 2:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [38]:
#exporting the model 
file = 'trained_model.sav'
pickle.dump(pipeline1, open(file, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.